GC Net Data

In [2]:
!pip install pyarrow

You should consider upgrading via the '/Users/nilsfulde/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [1]:
!pip install git+https://github.com/GEUS-PROMICE/pyNEAD.git

  Cloning https://github.com/GEUS-PROMICE/pyNEAD.git to c:\users\mabj16ac\appdata\local\temp\4\pip-req-build-1zu17gro
  Resolved https://github.com/GEUS-PROMICE/pyNEAD.git to commit d811220c2d048d84d4612b045880d6ca914c39a4
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for nead: filename=nead-0.0.0-py2.py3-none-any.whl size=10663 sha256=4736ab4fc8e3fe62dc67c2c15b6c58ff60aed7a9afae1d2942330122de5b691b
  Stored in directory: C:\Users\mabj16ac\AppData\Local\Temp\4\pip-ephem-wheel-cache-zymeu2hk\wheels\7e\b5\9c\bf1fb23ef32c0fd131d467e2ee10e1bc483361a4dba16b3600
Successfully built nead


  Running command git clone --filter=blob:none --quiet https://github.com/GEUS-PROMICE/pyNEAD.git 'C:\Users\mabj16ac\AppData\Local\Temp\4\pip-req-build-1zu17gro'


In [2]:
import os
print(os.getcwd())
#os.chdir('c:\\Users\\mabj16ac\\Desktop\\Thesis\\GEUS-Master-Thesis\\scripts')
os.chdir('/Users/nilsfulde/Desktop/Master_Thesis/scripts')
os.chdir("../")

/Users/nilsfulde/Desktop/Master_Thesis/scripts


In [20]:
#!/usr/bin/env bash

# Latest L1 data: https://github.com/GEUS-Glaciology-and-Climate/GC-Net-level-1-data-processing/tree/main/L1
# API contents of latest L1 data (raw URLs etc.): https://api.github.com/repositories/319306521/contents/L1

import os
os.getcwd()
#os.chdir('C:\\Users\\nifu18ab\\Desktop\\GEUS-Master-Thesis')
os.chdir('c:\\Users\\mabj16ac\\Desktop\\Thesis\\GEUS-Master-Thesis\\scripts')
os.chdir("../")
print(os.getcwd())


try:
    os.mkdir("data")
    os.mkdir("data/gc_net/raw_data/data_daily")
    os.mkdir("data/gc_net/raw_data/data_hourly")
except:
    print('Overwritting existing data in "/data"')

import urllib.request

# Download data
print("Downloading daily data...\r")


# xargs -n 1 curl --silent -O --output-dir data_daily < ../metadata/urls_1.txt
for url in open("metadata/urls_1.txt"):
    # Split on the rightmost / and take everything on the right side of that
    name = url.rsplit("/", 1)[-1].replace("\r", "")
    # Strip /n at the end of filename
    name = name.strip()
    # Combine the name and the downloads directory to get the local filename
    filename = os.path.join("data/gc_net/raw_data/data_daily", name)

    # Download the file if it does not exist
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(url, filename)


c:\Users\mabj16ac\Desktop\Thesis\GEUS-Master-Thesis
Overwritting existing data in "/data"


In [21]:
print("Downloading hourly data...\r")
# xargs -n 1 curl --silent -O --output-dir data_hourly < ../metadata/urls_2.txt
for url in open("metadata/urls_2.txt"):
    # Split on the rightmost / and take everything on the right side of that
    name = url.rsplit("/", 1)[-1].replace("\r", "")
    # Strip /n at the end of filename
    name = name.strip()
    # Combine the name and the downloads directory to get the local filename
    filename = os.path.join("data/gc_net/raw_data/data_hourly", name)

    # Download the file if it does not exist
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(url, filename)

In [33]:
import pandas as pd
import nead

def process_gcnet_daily():
    print('PROCESSING GC_NET DAILY...')
    # Convert NEAD files to Pandas dataframes
    station = pd.read_csv("metadata/station_info.csv", header=0)
    dfs_daily = []

    for name, ID in zip(station.Name, station.ID):
        format_name = name.replace(" ", "")
        files = "data/gc_net/raw_data/data_daily/" + str(ID).zfill(2) + "-" + format_name + "_daily.csv"
        ds_daily = nead.read(files, index_col=0)
        df_daily = ds_daily.to_dataframe()
        df_daily.insert(
            loc=0, column="station_name", value=name
        )  # Add station_name column to each dataframe
        dfs_daily.append(df_daily)

    # Concatenate dataframes
    df_daily = pd.concat(dfs_daily).sort_index()

    # Delete irrelevant columns from dataframe (i.e. null columns and flag columns)
    # null_columns = df_daily.columns[df_daily.isnull().all()]
    # flag_columns = df_daily.filter(regex="flag$").columns
    # print(null_columns)
    # print(flag_columns)

    df_daily = df_daily.drop(
        columns=[
            "OSWR_max",
            "HW2_adj_flag",
            "P_adj_flag",
            "HW1_adj_flag",
            "OSWR_adj_flag",
            "HS1_adj_flag",
            "HS2_adj_flag",
            "TA3_adj_flag",
            "TA4_adj_flag",
            "DW1_adj_flag",
        ]
    )
    # Add season column to dataframe
    seasons = {
        1: "Winter",
        2: "Winter",
        3: "Spring",
        4: "Spring",
        5: "Spring",
        6: "Summer",
        7: "Summer",
        8: "Summer",
        9: "Autumn",
        10: "Autumn",
        11: "Autumn",
        12: "Winter",
    }

    # Extract the month from the index and use the dictionary to map it to the corresponding season
    df_daily["season"] = df_daily.index.month.map(seasons)

    # Add year column to dataframe
    df_daily["year"] = df_daily.index.strftime("%Y")

    # Add month column to dataframe
    df_daily["month"] = df_daily.index.strftime("%B")

    # Rename Index Column to Datetime
    df_daily = df_daily.reset_index(inplace=False)
    df_daily = df_daily.rename(columns={'timestamp': 'Datetime'}, inplace=False) 

    # Rename 'station_name' to file
    df_daily = df_daily.rename(columns={'station_name': 'file'}, inplace=False)

    # Add Day of Year & Day of Century 
    df_daily['DayOfYear'] = df_daily['Datetime'].dt.dayofyear 
    df_daily['DayOfCentury'] = df_daily['Datetime'].dt.dayofyear+365*(df_daily['Datetime'].dt.year-1)


    # # Add day column to dataframe
    # df_daily["day"] = df_daily.index.strftime("%d")

    # Add hour column to dataframe
    # df_daily["hour"] = df_daily.index.strftime("%h")

    # Change column headers
    #header = pd.read_csv('/content/drive/MyDrive/Master_Thesis/metadata/Masterdata_GCNET.csv', sep = ";")
    header = pd.read_csv('metadata/Masterdata_GCNET.csv', sep = ";")
    df_daily = df_daily.rename(columns = header.set_index('fields')['display_description'])

    print('SAVING PROCESSED GC_NET_DAILY TO CSVs...')
    counter = 1
    for station in df_daily['file'].unique():
        station_csv = df_daily.loc[df_daily['file'] == station]
        station_csv.to_csv('data/gc_net/daily_data/{num}_{station}.csv'.format(num=counter,station=station))
        counter += 1 
    print('FINISHED SAVING GC_NET DAILY TO CSVs')
    return df_daily

def process_gcnet_hourly():
    print('PROCESSING GC_NET HOURLY...')
    station = pd.read_csv("metadata/station_info.csv", header=0)
    dfs_hourly = []

    for name, ID in zip(station.Name, station.ID):
        format_name = name.replace(" ", "")
        files = "data/gc_net/raw_data/data_hourly/" + str(ID).zfill(2) + "-" + format_name + ".csv"
        ds_hourly = nead.read(files, index_col=0)
        df_hourly = ds_hourly.to_dataframe()
        df_hourly.insert(
            loc=0, column="station_name", value=name
        )  # Add station_name column to each dataframe
        dfs_hourly.append(df_hourly)

    # Concatenate dataframes
    df_hourly = pd.concat(dfs_hourly).sort_index()

    # Delete irrelevant columns from dataframe (i.e. null columns and flag columns)
    # null_columns = df_hourly.columns[df_hourly.isnull().all()]
    # flag_columns = df_hourly.filter(regex="flag$").columns
    # print(null_columns)
    # print(flag_columns)

    df_hourly = df_hourly.drop(
        columns=[
            "OSWR_max",
            "HW2_adj_flag",
            "P_adj_flag",
            "HW1_adj_flag",
            "OSWR_adj_flag",
            "HS1_adj_flag",
            "HS2_adj_flag",
            "TA3_adj_flag",
            "TA4_adj_flag",
            "DW1_adj_flag",
        ]
    )
    # Add year column to dataframe
    df_hourly["year"] = df_hourly.index.strftime("%Y")

    # Add month column to dataframe
    df_hourly["month"] = df_hourly.index.strftime("%B")

    # Add season column to dataframe
    seasons = {
        1: "Winter",
        2: "Winter",
        3: "Spring",
        4: "Spring",
        5: "Spring",
        6: "Summer",
        7: "Summer",
        8: "Summer",
        9: "Autumn",
        10: "Autumn",
        11: "Autumn",
        12: "Winter",
    }

    # Extract the month from the index and use the dictionary to map it to the corresponding season
    df_hourly["season"] = df_hourly.index.month.map(seasons)


    #Rename Index Column to Datetime
    df_hourly = df_hourly.reset_index(inplace=False)
    df_hourly = df_hourly.rename(columns={'timestamp': 'Datetime'}, inplace=False) 

    # Rename 'station_name' to file
    df_hourly = df_hourly.rename(columns={'station_name': 'file'}, inplace=False)

    # Add Day of Year & Day of Century 
    df_hourly['DayOfYear'] = df_hourly['Datetime'].dt.dayofyear 
    df_hourly['DayOfCentury'] = df_hourly['Datetime'].dt.dayofyear+365*(df_hourly['Datetime'].dt.year-1)


    # # Add day column to dataframe
    # df_hourly["day"] = df_hourly.index.strftime("%d")

    # # Add hour column to dataframe
    # df_hourly["hour"] = df_hourly.index.strftime("%h")


    #header = pd.read_csv('/content/drive/MyDrive/Master_Thesis/metadata/Masterdata_GCNET.csv', sep = ";")
    header = pd.read_csv('metadata/Masterdata_GCNET.csv', sep = ";")
    df_hourly = df_hourly.rename(columns = header.set_index('fields')['display_description'])

    print('SAVING PROCESSED GC_NET HOURLY TO CSVs')
    counter = 1
    for station in df_hourly['file'].unique():
        station_csv = df_hourly.loc[df_hourly['file'] == station]
        station_csv.to_csv('data/gc_net/hourly_data/{num}_{station}.csv'.format(num=counter,station=station))
        counter += 1 
    print('FINISHED SAVING GC_NET HOURLY TO CSVs')
    

    # Save dataframe as parquet file
    #df_hourly.to_parquet("data/df_hourly.gzip", compression="gzip", engine='pyarrow')
    

In [32]:
process_gcnet_daily()
process_gcnet_hourly()


,Datetime,file,shortwave_incoming_radiation,shortwave_outgoing_radiation,net_radiation,air_temperature_1,air_temperature_1_max,air_temperature_1_min,air_temperature_cs500_air1,air_temperature_cs500_air2,...,incoming_uv_radiation,incoming_longwave_radiation,surface_temperature_1,surface_temperature_2,net_radiation_maximum,year,month,season,DayOfYear,DayOfCentury
0,1990-06-01 01:00:00,Swiss Camp 10m,NaN,NaN,NaN,1.72,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1990,June,Summer,152,726137
1,1990-06-01 02:00:00,Swiss Camp 10m,NaN,NaN,NaN,1.42,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1990,June,Summer,152,726137
2,1990-06-01 03:00:00,Swiss Camp 10m,NaN,NaN,NaN,0.88,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1990,June,Summer,152,726137
3,1990-06-01 04:00:00,Swiss Camp 10m,NaN,NaN,NaN,-0.13,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1990,June,Summer,152,726137
4,1990-06-01 05:00:00,Swiss Camp 10m,NaN,NaN,NaN,-1.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1990,June,Summer,152,726137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3967230,2022-10-16 22:00:00,Humboldt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022,October,Autumn,289,737954
3967231,2022-10-16 23:00:00,Humboldt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022,October,Autumn,289,737954
3967232,2022-10-17 00:00:00,Humboldt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022,October,Autumn,290,737955
3967233,2022-10-17 01:00:00,Humboldt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022,October,Autumn,290,737955


New Promice

In [17]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

def process_hourly_data(csv_directory="../../PROMICE-AWS-toolbox/out/test", add_meta_data=False):
        # List all CSV files in the directory
    csv_files = [f for f in os.listdir(csv_directory) if f.endswith('.csv')]

    # Combine all CSV files into a single DataFrame
    dfs = []
    for f in csv_files:
        df = pd.read_csv(os.path.join(csv_directory, f), index_col=False)
        df.insert(0, 'stid', f[:-7])
        dfs.append(df)
    df_hourly = pd.concat(dfs)

    #read metadata from promice repository
    station = pd.read_csv('../../PROMICE-AWS-toolbox/metadata/AWS_station_locations.csv', index_col=False)
    station.to_csv('../data/promice/new_promice/AWS_station_locations.csv', index=False)


    #display(output station and columns summary)
    print('Stations loaded:')
    display(df_hourly['stid'].unique())
    print('columns in dataset:')
    print(list(df_hourly.columns))
    print("FINISHED LOADING CSV's")

    # Add year column to dataframe
    df_hourly["Datetime"] = pd.to_datetime(df_hourly.time)
    df_hourly["Datetime"] = pd.to_datetime(df_hourly["Datetime"], format="%Y-%m-%d %H:%M:%S")

    #Rename Index Column to Datetime
    df_hourly = df_hourly.reset_index(inplace=False)

    if add_meta_data==True:
        add_nice_to_haves(df_hourly, 'hourly')

    header = pd.read_csv('../metadata/promice_header.csv', sep = ";")
    df_hourly = df_hourly.rename(columns = header.set_index('standard_name')['units']) #Units because headers are shifted
    print(df_hourly.columns)
    
    # This bit to detect Ablation
    ablation = pd.DataFrame() # Create an empty dataframe to store the filtered and modified dataframes
    
    # The next lines to format Datetime Correctly, assign it as index while still keeping it as a column
    df_hourly['index'] = df_hourly.index
    df_hourly["Datetime"] = pd.to_datetime(df_hourly["Datetime"], format="%Y-%m-%d %H:%M:%S")
    df_hourly = df_hourly.rename(columns={'Datetime': 'Index_Datetime'})
    df_hourly = df_hourly.set_index(['Index_Datetime'])
    df_hourly['Datetime'] = df_hourly.index
    df_hourly["Datetime"] = pd.to_datetime(df_hourly["Datetime"], format="%Y-%m-%d %H:%M:%S")
    
    for station in df_hourly["stid"].unique():
        print("Calculating Ablation" , station)
        df_stid = df_hourly[df_hourly['stid'] == station].copy()

        # defining ice ablation period   
        smoothed_PT =  df_stid['Depth of pressure transducer in ice - corrected'].interpolate(limit=72).rolling('14D', min_periods=1).mean().shift(-7*14/2, freq='h')
        smoothed_PT = smoothed_PT.rolling('14D', min_periods=1).mean().shift(-7*14/2, freq='h')

        threshold_ablation = -0.0002 # Modify value if needed
        ind_ablation = np.logical_and(smoothed_PT.diff().values < threshold_ablation, 
                                      np.isin(smoothed_PT.diff().index.month, [6, 7, 8, 9]))

        ind_ablation = np.concatenate((ind_ablation[4*24:], np.repeat(ind_ablation[-(4*24):-(4*24-1)], 4*24)))

        df_stid['Ablation'] = ind_ablation.tolist()
        df_stid['stid'] = station # Add a column for the current station
        ablation = ablation.append(df_stid[['Datetime', 'stid', 'Ablation']])
    
    
    df_hourly = df_hourly.set_index(['index'])
    
    # Merge the new column with the original dataframe based on datetime and station
    df_hourly = pd.merge(df_hourly, ablation, on=['Datetime', 'stid'], how='left')
        
    # This bit to add a beginning, middle & end of melting season attribute
    beginning = 0.15
    end = 0.85

    melting_season = pd.DataFrame() # Create an empty dataframe to store the filtered and modified dataframes


    for station in df_hourly["stid"].unique():

        print("Calculating Melting Season" , station)
        df_stid = df_hourly[df_hourly['stid'] == station].copy() 

        # Iterate over each year
        for year in df_stid['Datetime'].dt.year.unique():

            print("   Calculating" , year)

            # Get the dataframe for the current year
            year_df = df_stid[df_stid['Datetime'].dt.year == year]

            # Determine the threshold values for the beginning and end of the melting season
            ablation_df = year_df[year_df['Ablation'] == True]
            lower_threshold = ablation_df['Datetime'].dt.dayofyear.quantile(beginning)
            upper_threshold = ablation_df['Datetime'].dt.dayofyear.quantile(end)

            # Set the Melting Season value for each row in the current year
            for index, row in year_df.iterrows():

                if row['Ablation'] == True:
                    if row['Datetime'].dayofyear <= lower_threshold:
                        df_stid.loc[index, 'Melting Season'] = 'beginning'
                    elif row['Datetime'].dayofyear >= upper_threshold:
                        df_stid.loc[index, 'Melting Season'] = 'end'
                    else:
                        df_stid.loc[index, 'Melting Season'] = 'middle'
                        
                elif (row['Datetime'] - pd.Timedelta(days=30)).dayofyear <= lower_threshold:
                    df_stid.loc[index, 'Melting Season'] = 'pre'
                else:
                    df_stid.loc[index, 'Melting Season'] = ''

        df_stid['stid'] = station # Add a column for the current station
        melting_season = melting_season.append(df_stid[['Datetime', 'stid', 'Melting Season']])

    # Merge the new column with the original dataframe based on datetime and station
    df_hourly = pd.merge(df_hourly, melting_season, on=['Datetime', 'stid'], how='left')
    
    # Fill NAN values
    df_hourly['Melting Season'] = df_hourly['Melting Season'].fillna('')
    
    #This bit to add delta column 
    delta = pd.DataFrame() # Create an empty dataframe to store the filtered and modified dataframes
    df_hourly['Date'] = df_hourly['Datetime'].dt.date # Date to merge on
    
    for station in df_hourly["stid"].unique():
        print("Calculating", station)
        df_stid = df_hourly[df_hourly['stid'] == station].copy()
        df_stid['Date'] = df_stid['Datetime'].dt.date # extract date component
        df_stid['Surface height from combined measurements DELTA'] = df_stid.groupby('Date')['Surface height from combined measurements'].transform(lambda x: (x.iloc[-1] - x.iloc[0]))
        #df_stid['Surface height from combined measurements DELTA'] = df_stid['Surface height from combined measurements'].diff()
        df_stid['stid'] = station # Add a column for the current station
        delta = delta.append(df_stid[['Datetime', 'stid', 'Surface height from combined measurements DELTA']])

    # Merge the new column with the original dataframe based on datetime and station
    df_hourly = pd.merge(df_hourly, delta, on=['Datetime', 'stid'], how='left')

    
    # Remove not needed columns
    df_hourly = df_hourly[[column for column in df_hourly.columns if column not in exclude_list]]

    counter = 1
    for station in df_hourly['stid'].unique():
        station_csv = df_hourly.loc[df_hourly['stid'] == station]
        station_csv.to_csv('../data/promice/preprocessed/hourly/{station}.csv'.format(station=station))
        counter += 1 

    print("FINISHED PROCESSING HOURLY DATA")
    return df_hourly

def process_daily_data(dataframe=None, directory="../data/promice/preprocessed/hourly", add_meta_data=False):
    
    if not isinstance(dataframe, pd.DataFrame):
        # List all CSV files in the directory
        csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

        # Combine all CSV files into a single DataFrame
        dfs = []
        for f in csv_files:
            df = pd.read_csv(os.path.join(directory, f), index_col=False)
            dfs.append(df)
            df = pd.concat(dfs)
        print('finished loading csvs')
    else:
        df = dataframe.copy()
    
    # remove the Unnamed columns...
    df = df[[column for column in df.columns if column not in exclude_list]]
    
        #display(output station and columns summary)
    print('Stations loaded:')
    display(df['stid'].unique())
    print('columns in dataset:')
    print(list(df.columns))
    print("FINISHED LOADING CSV's")

    
    # Define the date column that you want to group by (replace "date_column" with the name of your column)
    min_values_per_day = 20
    
    # Group the data by weather station and date
    #grouped = df.groupby(['stid','date'])
    df["Datetime"] = pd.to_datetime(df["Datetime"], format="%Y-%m-%d")
    df['DayOfYear'] = df['Datetime'].dt.dayofyear
    df['DayOfCentury'] = ((df['Datetime'].dt.year - 1) * 365 + (df['Datetime'].dt.year - 1) // 4 + df['Datetime'].dt.dayofyear) % 365
    
    
    grouped = df.groupby(['stid','Datetime'])
    

    # Specify columns containing numerical values to be averaged
    columns_to_average = (df
                .select_dtypes(exclude=['object'])
                #.drop(columns=['index', 'DayOfYear', 'DayOfCentury'])
                .drop(columns=['DayOfYear', 'DayOfCentury', 'Datetime'])
                .columns
    )
    
    print("Columns to average:")
    print(columns_to_average)
    
    # Calculate the number of non-NaN values for each variable within each group
    valid_date_observations = grouped[columns_to_average].apply(lambda x: x.notna().sum() <= min_values_per_day)
    filter_excluded_columns = ['Albedo','Depth of pressure transducer in ice - corrected']
    valid_date_observations[filter_excluded_columns] = False


    # Calculate average per day per station
    df_filtered = grouped[columns_to_average].mean()

    #Remove means with less than 20 observations per day
    df_masked = df_filtered.mask(valid_date_observations, np.nan)

    df_daily = df_masked.reset_index().copy()
    
    print('columns in df_daily:')
    print(list(df_daily.columns))

    if add_meta_data==True:
        df_daily = add_nice_to_haves(df_daily, 'daily')

        
    #This bit to add delta column 
    delta = pd.DataFrame() # Create an empty dataframe to store the filtered and modified dataframes

    for station in df_daily["stid"].unique():
        print("Calculating", station)
        df_stid = df_daily[df_daily['stid'] == station].copy()
        df_stid['Surface height from combined measurements DELTA'] = df_stid['Surface height from combined measurements'].diff()
        df_stid['stid'] = station # Add a column for the current station
        delta = delta.append(df_stid[['Datetime', 'stid', 'Surface height from combined measurements DELTA']])

    # Merge the new column with the original dataframe based on datetime and station
    df_daily = pd.merge(df_daily, delta, on=['Datetime', 'stid'], how='left')
    
    df_daily = df_daily[[column for column in df_daily.columns if column not in exclude_list]]
    
    counter = 1
    for station in df_daily['stid'].unique():
        station_csv = df_daily.loc[df_daily['stid'] == station]
        station_csv.to_csv('../data/promice/preprocessed/daily/{station}.csv'.format(station=station))
        counter += 1 

    print("FINISHED PROCESSING HOURLY DATA")
    return df_daily

###########################################################################################
### Helper functions

def add_station_info(df):
    station = pd.read_csv('../data/promice/new_promice/AWS_station_locations.csv')
    station.rename(columns={'timestamp':'station_location_timestamp'})
    df.merge(station, how='left',on=['stid','stid'])
    return df

# helper functions for adding metadata like season, day, month, year, date, DayOfYear and DayOfCentury
def add_nice_to_haves(df, period):
    def add_season(df):
        seasons = {
            1: "Winter",
            2: "Winter",
            3: "Spring",
            4: "Spring",
            5: "Spring",
            6: "Summer",
            7: "Summer",
            8: "Summer",
            9: "Autumn",
            10: "Autumn",
            11: "Autumn",
            12: "Winter",
        }

        # Extract the month from the index and use the dictionary to map it to the corresponding season
        df["season"] = df['Datetime'].dt.month.map(seasons)
        return df

    def add_common(df):
        df["year"] = df['Datetime'].dt.strftime("%Y")
        df["month"] = df['Datetime'].dt.strftime("%B")
        df["date"] = df['Datetime'].dt.date
        df['DayOfYear'] = df['Datetime'].dt.dayofyear 
        df['DayOfCentury'] = df['Datetime'].dt.dayofyear+365*(df['Datetime'].dt.year-1)
        df = add_season(df)
        return df
        
    def add_day(df):
        # Add day column to dataframe
        df["day"] = df['Datetime'].dt.strftime("%d")
        return df

    def add_hour(df):
        # Add hour column to dataframe
        df["hour"] = df['Datetime'].dt.strftime("%h")
        return df

    if period == 'hourly':
        df = add_hour(
            add_day(
            add_common(df)))

    elif period == 'daily':
        df = add_day(
            add_common(df))
    elif period == 'monthly':
        df = add_common(df)
    
    return df 


In [18]:
def process_daily_data(dataframe=None, directory="../data/promice/preprocessed/hourly", add_meta_data=False):
    
    if not isinstance(dataframe, pd.DataFrame):
        # List all CSV files in the directory
        csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

        # Combine all CSV files into a single DataFrame
        dfs = []
        for f in csv_files:
            df = pd.read_csv(os.path.join(directory, f), index_col=False)
            dfs.append(df)
            df = pd.concat(dfs)
        print('finished loading csvs')
    else:
        df = dataframe.copy()
    
    # remove the Unnamed columns...
    df = df[[column for column in df.columns if column not in exclude_list]]
    
        #display(output station and columns summary)
    print('Stations loaded:')
    display(df['stid'].unique())
    print('columns in dataset:')
    print(list(df.columns))
    print("FINISHED LOADING CSV's")

    # Convert 'Datetime' column to datetime format
    df['Datetime'] = pd.to_datetime(df['Datetime'])

    # Group by 'stid' and daily frequency
    daily_groups = df.groupby(['stid', pd.Grouper(key='Datetime', freq='D')])

    # Define aggregation functions
    agg_functions = {'Air pressure (upper boom)': lambda x: x.mean() if len(x) > 20 else np.nan,
                     'Air temperature (upper boom)': lambda x: x.mean() if len(x) > 20 else np.nan,
                     'Relative humidity (upper boom) - corrected': lambda x: x.mean() if len(x) > 20 else np.nan,
                     'Specific humidity (upper boom)': lambda x: x.mean() if len(x) > 20 else np.nan,
                     'Wind speed (upper boom)': lambda x: x.mean() if len(x) > 20 else np.nan,
                     'Wind from direction (upper boom)': lambda x: x.mean() if len(x) > 20 else np.nan,
                     'Downwelling shortwave radiation - corrected': lambda x: x.mean() if len(x) > 20 else np.nan,
                     'Upwelling shortwave radiation - corrected': lambda x: x.mean() if len(x) > 20 else np.nan,
                     'Downwelling longwave radiation': lambda x: x.mean() if len(x) > 20 else np.nan,
                     'Upwelling longwave radiation': lambda x: x.mean() if len(x) > 20 else np.nan,
                     'Surface temperature': lambda x: x.mean() if len(x) > 20 else np.nan,
                     'Latent heat flux (upper boom)': lambda x: x.mean() if len(x) > 20 else np.nan,
                     'Sensible heat flux (upper boom)': lambda x: x.mean() if len(x) > 20 else np.nan,
                     'Surface height from combined measurements': 'mean',
                     'Surface height from combined measurements DELTA': 'mean',
                     'Albedo': 'mean',
                     'Cloud cover': lambda x: x.mean() if len(x) > 20 else np.nan,
                     'Ablation': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan,
                     'Melting Season': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan}

    # Apply aggregation functions to each group
    df_daily = daily_groups.agg(agg_functions)

    # Remove the multi-level index and reset the index
    df_daily.reset_index(inplace=True)

    
    print('columns in df_daily:')
    print(list(df_daily.columns))

    if add_meta_data==True:
        df_daily = add_nice_to_haves(df_daily, 'daily')

    
    df_daily = df_daily[[column for column in df_daily.columns if column not in exclude_list]]
    
    counter = 1
    for station in df_daily['stid'].unique():
        station_csv = df_daily.loc[df_daily['stid'] == station]
        station_csv.to_csv('../data/promice/preprocessed/daily/{station}.csv'.format(station=station))
        counter += 1 

    print("FINISHED PROCESSING HOURLY DATA")
    return df_daily

In [19]:
# This bit to include only relevant features
exclude_list = [                                                   # 'index', # excluded because of unimportant information
                                                                   #  'stid', # excluded because of unimportant information
#                                                'Air pressure (upper boom)',
#                                             'Air temperature (upper boom)',
                                            'Relative humidity (upper boom)', # excluded because of corrected feature
#                               'Relative humidity (upper boom) - corrected',
#                                           'Specific humidity (upper boom)',
#                                                  'Wind speed (upper boom)',
#                                         'Wind from direction (upper boom)',
                                          'Downwelling shortwave radiation', # excluded because of corrected feature
#                              'Downwelling shortwave radiation - corrected',
                                            'Upwelling shortwave radiation', # excluded because of corrected feature
#                                'Upwelling shortwave radiation - corrected',
#                                                                   'Albedo', 
#                                           'Downwelling longwave radiation',
#                                             'Upwelling longwave radiation',
#                                                              'Cloud cover',
#                                                      'Surface temperature',
#                                            'Latent heat flux (upper boom)', 
#                                          'Sensible heat flux (upper boom)', 
                                                        'Upper boom height', # excluded because of unimportant information
                                                             'Stake height', # excluded due to missing values
                                      'Depth of pressure transducer in ice', # excluded due to derived correlation with y
                          'Depth of pressure transducer in ice - corrected', # excluded due to derived correlation with y
                   'Precipitation (upper boom) (cumulative solid & liquid)', # excluded because of corrected feature
       'Precipitation (upper boom) (cumulative solid & liquid) – corrected', # excluded due to missing values
                                              'Ice temperature at sensor 1', # excluded due to missing values
                                              'Ice temperature at sensor 2', # excluded due to missing values
                                              'Ice temperature at sensor 3', # excluded due to missing values
                                              'Ice temperature at sensor 4', # excluded due to missing values
                                              'Ice temperature at sensor 5', # excluded due to missing values
                                              'Ice temperature at sensor 6', # excluded due to missing values
                                              'Ice temperature at sensor 7', # excluded due to missing values
                                              'Ice temperature at sensor 8', # excluded due to missing values
                                                             'Tilt to east', # excluded because of unimportant information 
                                                            'Tilt to north', # excluded because of unimportant information
                                         'Station rotation from true North', # excluded because of unimportant information
                                                                 'Latitude', # excluded because of unimportant information
                                                                'Longitude', # excluded because of unimportant information
                                                                 'Altitude', # excluded because of unimportant information
                                                                 'GPS time', # excluded because of unimportant information 
                               'Height of EGM96 geoid over WGS84 ellipsoid', # excluded because of unimportant information
                                                                  'GeoUnit', # excluded because of unimportant information
                             'GPS horizontal dillution of precision (HDOP)', # excluded because of unimportant information
                                                 'GPS number of satellites', # excluded because of unimportant information
                                                                  'Quality', # excluded because of unimportant information
                                                          'Battery voltage', # excluded because of unimportant information
                                                                      'nan',
                                           'Battery voltage (sample start)', # excluded because of unimportant information
                                                 'Fan current (upper boom)', # excluded because of unimportant information
                       'Frequency of vibrating wire in precipitation gauge', # excluded because of unimportant information
                                                       'Logger temperature', # excluded because of unimportant information
                                             'Radiation sensor temperature', # excluded because of unimportant information
                                         'latitude from modem (email text)', # excluded because of unimportant information
                                        'longitude from modem (email text)', # excluded because of unimportant information
                                                         'Surface height 1', # excluded due to derived correlation with y
                                                         'Surface height 1', # excluded due to derived correlation with y
                                                        'z_surf_1_adj_flag', # excluded due to derived correlation with y
                                                        'z_surf_2_adj_flag', # excluded due to derived correlation with y
#                                'Surface height from combined measurements', 
                                   'Depth of ice temperature measurement 1', # excluded because of unimportant information
                                   'Depth of ice temperature measurement 2', # excluded because of unimportant information
                                   'Depth of ice temperature measurement 3', # excluded because of unimportant information
                                   'Depth of ice temperature measurement 4', # excluded because of unimportant information
                                   'Depth of ice temperature measurement 5', # excluded because of unimportant information
                                   'Depth of ice temperature measurement 6', # excluded because of unimportant information
                                   'Depth of ice temperature measurement 7', # excluded because of unimportant information
                                   'Depth of ice temperature measurement 8', # excluded because of unimportant information
                               'Ice temperature interpolated at 10 m depth', # excluded due to missing values
                                                'Air pressure (lower boom)', # excluded due to missing values
                                             'Air temperature (lower boom)', # excluded due to missing values
                                           'Relative humidity (lower boom)', # excluded due to missing values
                               'Relative humidity (lower boom) - corrected', # excluded due to missing values
                                           'Specific humidity (lower boom)', # excluded due to missing values
                                                  'Wind speed (lower boom)', # excluded due to missing values
                                         'Wind from direction (lower boom)', # excluded due to missing values
                                            'Latent heat flux (lower boom)', # excluded due to missing values
                                          'Sensible heat flux (lower boom)', # excluded due to missing values
                                                        'Lower boom height', # excluded due to missing values
                   'Precipitation (lower boom) (cumulative solid & liquid)', # excluded due to missing values
       'Precipitation (lower boom) (cumulative solid & liquid) – corrected', # excluded due to missing values
                                              'Ice temperature at sensor 9', # excluded due to missing values
                                             'Ice temperature at sensor 10', # excluded due to missing values
                                             'Ice temperature at sensor 11', # excluded due to missing values
                                                 'Fan current (lower boom)', # excluded due to missing values
                                   'Depth of ice temperature measurement 9', # excluded due to missing values
                                  'Depth of ice temperature measurement 10', # excluded due to missing values
                                  'Depth of ice temperature measurement 11', # excluded due to missing values
                                                        'z_pt_cor_adj_flag', # excluded due to derived correlation with y
                              'Ice surface height adjusted for maintenance', # excluded due to derived correlation with y
                                'Surface height adjusted for maintenance 1', # excluded due to derived correlation with y
                                  'Surface height adjusted for maintenance', # excluded due to derived correlation with y
#                                                                 'Datetime',
#                                                                 'Ablation',  
#                          'Surface height from combined measurements DELTA'
                                                              'Unnamed: 18',# excluded because we do not know what it is
                                                                'month_num',# excluded because we do not know what it is
                                                                     'date',# excluded because we do not know what it is
                                                       'Surface height 1.1',# excluded because we do not know what it is
                                                              'Unnamed: 48',# excluded because we do not know what it is
                                                                     'Time',# excluded because we do not know what it is
                                                               'Unnamed: 0',
                                                              'Unnamed: 17',
                                                                     'Date',# excluded because we do not know what it is
                                                       
               ]

In [20]:
#os.chdir('c:\\Users\\mabj16ac\\Desktop\\Thesis\\GEUS-Master-Thesis\\scripts')
os.chdir('/Users/nilsfulde/Desktop/Master_Thesis/scripts')

#header = pd.read_csv('../metadata/promice_header.csv', sep = ";")
#df_hourly = df_hourly.rename(columns = header.set_index('standard_name')['units'])
df_hourly = process_hourly_data(add_meta_data=False)
#df_hourly = df_hourly.rename(columns = header.set_index('standard_name')['units'])

#df_hourly = process_hourly_data(directory="../data/new_promice/all_promice_data.parquet.gzip")
############################################

df_daily = process_daily_data(add_meta_data=False)
#df_daily = df_daily.rename(columns = header.set_index('standard_name')['units'])

#df_daily = process_daily_data(directory="../data/new_promice/all_promice_data_hourly.parquet.gzip", add_meta_data=True)

############################################
#df_monthly = process_monthly_data(dataframe=df_daily, add_meta_data=True)
#df_monthly = df_monthly.rename(columns = header.set_index('standard_name')['units'])

#df_monthly = process_monthly_data(directory='..\\Data\\new_promice\\all_promice_data_daily.parquet.gzip', add_meta_data=True)

### helper function to add station information such as # of booms, location and classification

# df_hourly = add_station_info(df_hourly)
# df_daily = add_station_info(df_daily)
# df_monthly = add_station_info(df_monthly)

Stations loaded:


array(['KAN_L', 'SCO_L'], dtype=object)

columns in dataset:
['stid', 'time', 'p_u', 't_u', 'rh_u', 'rh_u_cor', 'qh_u', 'wspd_u', 'wdir_u', 'dsr', 'dsr_cor', 'usr', 'usr_cor', 'albedo', 'dlr', 'ulr', 'cc', 't_surf', 'dlhf_u', 'dshf_u', 'z_boom_u', 'z_stake', 'z_pt', 'z_pt_cor', 'precip_u', 'precip_u_cor', 't_i_1', 't_i_2', 't_i_3', 't_i_4', 't_i_5', 't_i_6', 't_i_7', 't_i_8', 'tilt_x', 'tilt_y', 'rot', 'gps_lat', 'gps_lon', 'gps_alt', 'gps_time', 'gps_geoid', 'gps_geounit', 'gps_hdop', 'gps_numsat', 'gps_q', 'batt_v', 'batt_v_ini', 'batt_v_ss', 'fan_dc_u', 'freq_vw', 't_log', 't_rad', 'msg_lat', 'msg_lon', 'z_pt_cor_adj_flag', 'z_surf_1', 'z_surf_2', 'z_surf_2_adj_flag', 'z_pt_cor_adj', 'z_surf_1_adj', 'z_surf_2_adj', 'z_surf_combined', 'depth_t_i_1', 'depth_t_i_2', 'depth_t_i_3', 'depth_t_i_4', 'depth_t_i_5', 'depth_t_i_6', 'depth_t_i_7', 'depth_t_i_8', 't_i_10m']
FINISHED LOADING CSV's
Index([                                                             'index',
                                                               

array(['SCO_L', 'KAN_L'], dtype=object)

columns in dataset:
['stid', 'Air pressure (upper boom)', 'Air temperature (upper boom)', 'Relative humidity (upper boom) - corrected', 'Specific humidity (upper boom)', 'Wind speed (upper boom)', 'Wind from direction (upper boom)', 'Downwelling shortwave radiation - corrected', 'Upwelling shortwave radiation - corrected', 'Albedo', 'Downwelling longwave radiation', 'Upwelling longwave radiation', 'Cloud cover', 'Surface temperature', 'Latent heat flux (upper boom)', 'Sensible heat flux (upper boom)', 'Surface height from combined measurements', 'Datetime', 'Ablation', 'Melting Season', 'Surface height from combined measurements DELTA']
FINISHED LOADING CSV's
columns in df_daily:
['stid', 'Datetime', 'Air pressure (upper boom)', 'Air temperature (upper boom)', 'Relative humidity (upper boom) - corrected', 'Specific humidity (upper boom)', 'Wind speed (upper boom)', 'Wind from direction (upper boom)', 'Downwelling shortwave radiation - corrected', 'Upwelling shortwave radiation - correc

In [ ]:
def promice_pull(directory='/data/promice/new_promice/daily_data'):
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

    # Combine all CSV files into a single DataFrame
    dfs = []
    for f in csv_files:
        df = pd.read_csv(os.path.join(directory, f), index_col=False)
        df.insert(0, 'stid', f[:-7])
        dfs.append(df)
    df = pd.concat(dfs)
    return df

In [ ]:
C:\Users\mabj16ac\Desktop\Thesis\GEUS-Master-Thesis\data\promice\new_promice\hourly_data
null_columns = df_hourly.columns[df_hourly.isnull().all()]
flag_columns = df_hourly.filter(regex="flag$").columns
print("null columns:\n", null_columns)
print("flag_columns:\n", flag_columns)

In [ ]:
df_monthly.info

Promice Data

In [ ]:
!wget -r -e robots=off -nH --cut-dirs=3 --content-disposition "https://dataverse.geus.dk/api/datasets/:persistentId/dirindex?persistentId=doi:10.22008/FK2/8SS7EW"

In [9]:
#############################################
# Redundant

def process_monthly_data(dataframe=None, directory="", add_meta_data=False):

    if not isinstance(dataframe, pd.DataFrame):
        df = pd.read_parquet(directory)
    else:
        df = dataframe.copy()

    df['month_year'] = df['Datetime'].dt.to_period('M')
    # Define the date column that you want to group by (replace "date_column" with the name of your column)
    min_values_per_month = 24

    # Create a new column with the month and year of the date column

    # Group the data by weather station and date
    grouped = df.groupby(['stid','month_year'])

    # Specify columns containing numerical values to be averaged
    columns_to_average = (df
                .select_dtypes(exclude=['object'])
                .drop(columns=['Datetime', 'DayOfYear', 'DayOfCentury'])
                .columns
    )

    # Calculate the number of non-NaN values for each variable within each group
    valid_date_observations = grouped[columns_to_average].apply(lambda x: x.notna().sum() <= min_values_per_month)

    # Calculate average per day per station
    df_filtered = grouped[columns_to_average].mean()

    #Remove means with less than 20 observations per day
    df_masked = df_filtered.mask(valid_date_observations, np.nan)

    df_monthly = df_masked.reset_index().copy()

    df_monthly["Datetime"] = pd.to_datetime(df_monthly['month_year'].astype(str) + '-01')

    if add_meta_data==True:
        df_monthly = add_nice_to_haves(df_monthly, 'monthly')

    #display(df_monthly)

    # Convert the DataFrame to a compressed Parquet file
    output_file = "..\\Data\\new_promice\\all_promice_data_monthly.parquet.gzip"
    df_monthly.to_parquet(output_file, compression='gzip', engine='pyarrow')

    print('FINISHED PROCESSING MONTHLY DATA')
    return df_monthly

In [10]:
# save daily data 
# Get the files from the path provided in the OP
files = Path(path).glob('*_day_v03.txt')  # .rglob to get subdirectories

dfs = list()
for f in files:
    data = pd.read_csv(f, delimiter=r"\s+", engine='python')
    # .stem is method for pathlib objects to get the filename w/o the extension
    data['file'] = f.stem
    dfs.append(data)

df = pd.concat(dfs, ignore_index=True)

df = df.replace(-999, np.nan)

df["Month"] = df["MonthOfYear"]
df["Day"] = df["DayOfMonth"]

df["Datetime"] = pd.to_datetime(df[["Year", "Month", "Day"]], format='%Y%m%d')

# Save dataframe as parquet file
df.to_parquet("data/promice_daily.gzip", compression="gzip", engine='pyarrow')
#df.to_parquet("/content/drive/MyDrive/Master_Thesis/data/promice_daily.gzip", compression="gzip")

In [11]:
# save monthly data 
# Get the files from the path provided in the OP
files = Path(path).glob('*_month_v03.txt')  # .rglob to get subdirectories

dfs = list()
for f in files:
    data = pd.read_csv(f, delimiter=r"\s+", engine='python')
    # .stem is method for pathlib objects to get the filename w/o the extension
    data['file'] = f.stem
    dfs.append(data)

df = pd.concat(dfs, ignore_index=True)

df = df.replace(-999, np.nan)

df["Month"] = df["MonthOfYear"]

df["Datetime"] = pd.to_datetime(['{}-{}-01'.format(y, m) for y, m in zip(df.Year, df.Month)])

# Save dataframe as parquet file
df.to_parquet("data/promice_monthly.gzip", compression="gzip", engine='pyarrow')
#df.to_parquet("/content/drive/MyDrive/Master_Thesis/data/promice_monthly.gzip", compression="gzip")